In [1]:
import os
from pyspark.sql import SparkSession

# ⭐ 关键更新: 适用于 Spark 3.3.x 和 Scala 2.12 的 Snowflake Connector
SNOWFLAKE_CONNECTOR_PACKAGE = "net.snowflake:spark-snowflake_2.12:2.11.0-spark_3.3"
# 注：如果遇到问题，您可以尝试更新版本，例如 2.12.0 或更高版本。

def create_spark(app_name="5400-news-elt-snowflake"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.RawLocalFileSystem")
        .config("spark.hadoop.fs.file.impl.disable.cache", "true")
        # ⭐ 确保依赖正确
        .config("spark.jars.packages", SNOWFLAKE_CONNECTOR_PACKAGE)
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark

In [2]:
# -------------------------------------------------------------------------
# Snowflake 连接配置 (必须替换为您的真实信息！)
# -------------------------------------------------------------------------
SNOWFLAKE_OPTIONS = {
    "sfURL": "pobriux-fyc84817.snowflakecomputing.com",
    "sfUser": "AKAAAAFK",
    "sfAuthenticator": "externalbrowser",
    "sfRole": "ACCOUNTADMIN",
    "sfWarehouse": "SNOWFLAKE_LEARNING_WH",
    "sfDatabase": "SNOWFLAKE_LEARNING_DB",
    "sfSchema": "PUBLIC",
}


def save_to_snowflake(df, table_name, mode="overwrite"):
    """
    使用 Spark Connector 将 DataFrame 写入 Snowflake
    """
    print(f"❄️ 正在尝试将数据写入 Snowflake 表: {table_name}...")

    try:
        # 使用 .format("net.snowflake.spark.snowflake") 调用连接器
        (
            df.write
            .format("net.snowflake.spark.snowflake")
            .options(**SNOWFLAKE_OPTIONS) # 传入连接配置
            .option("dbtable", table_name) # 目标表名
            .mode(mode)
            .save()
        )
        print(f"✅ 数据已成功写入 Snowflake 表: {table_name}")
    except Exception as e:
        print(f"❌ Snowflake 写入失败。请检查您的凭证、账号URL和网络连接。")
        print(f"错误详情: {e}")

In [2]:
import os
from pyspark.sql import functions as F
from pyspark.sql import types as T

BASE = r"D:\Columbia\Fall2025\5400\project\layer"
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

BRONZE = os.path.join(BASE, "bronze")
SILVER = os.path.join(BASE, "silver")
GOLD = os.path.join(BASE, "gold")
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")
# 路径定义
BASE = r"D:\Columbia\Fall2025\5400\project\layer"
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

BRONZE = os.path.join(BASE, "bronze")
SILVER = os.path.join(BASE, "silver")
GOLD = os.path.join(BASE, "gold")
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")
SENTIMENT_OUTPUT = os.path.join(BASE, "sentiment_output") # 对应流程图中的 sentiment.parquet → S3 NLP Zone
FACT_NEWS_SENTIMENT = os.path.join(BASE, "fact_news_sentiment") # 对应流程图中的 FACT_NEWS_SENTIMENT → Snowflake

In [3]:
def build_bronze(spark):
    os.makedirs(BRONZE, exist_ok=True)

    print("📥 Loading CSV...")
    df = (
        spark.read.format("csv")
        .options(
            header="true",
            inferSchema="true",
            multiLine="true",
            escape="\"",
            quote="\"",
            mode="PERMISSIVE"
        )
        .load(CSV_PATH)
    )

    # Cleaning string columns
    for c, t in df.dtypes:
        if t == "string":
            df = df.withColumn(c, F.trim(F.col(c)))

    # date
    if "Date" in df.columns:
        df = df.withColumn("Date", F.to_date("Date", "yyyy-MM-dd"))

    df = df.repartition(400)

    print("💾 Writing Bronze...")
    df.write.mode("overwrite").parquet(BRONZE)
    print("🥉 Bronze Ready:", BRONZE)

    return df

def build_silver(spark):
    os.makedirs(SILVER, exist_ok=True)
    df = spark.read.parquet(BRONZE)

    # key fields not null
    if "Date" in df.columns:
        df = df.filter(F.col("Date").isNotNull())
    if "Stock_symbol" in df.columns:
        df = df.filter(F.col("Stock_symbol").isNotNull())

    # drop duplicates
    df = df.dropDuplicates()

    # standardize text fields
    if "Publisher" in df.columns:
        df = df.withColumn("Publisher_norm", F.upper("Publisher"))
    if "Author" in df.columns:
        df = df.withColumn("Author_norm", F.upper("Author"))

    # unique ID
    df = df.withColumn("news_id", F.monotonically_increasing_id())

    # repartition
    df = df.repartition(400, "Stock_symbol", "Date")

    print("💾 Writing Silver...")
    df.write.mode("overwrite").parquet(SILVER)
    print("🥈 Silver Ready:", SILVER)

    return df

def build_gold(spark):
    os.makedirs(GOLD, exist_ok=True)
    df = spark.read.parquet(SILVER)

    df = df.withColumn("title_len", F.length("Article_title"))

    agg_cols = [
        F.count("*").alias("article_count"),
        F.countDistinct("Publisher").alias("publisher_count"),
        F.avg("title_len").alias("avg_title_len"),
    ]

    for col in ["Textrank_summary", "Lsa_summary", "Luhn_summary", "Lexrank_summary"]:
        if col in df.columns:
            agg_cols.append(F.first(col, ignorenulls=True).alias("sample_summary"))
            break

    gold_df = df.groupBy("Date", "Stock_symbol").agg(*agg_cols)
    gold_df = gold_df.repartition(200, "Stock_symbol")

    print("💾 Writing Gold...")
    gold_df.write.mode("overwrite").parquet(GOLD)
    print("🥇 Gold Ready:", GOLD)

    return gold_df

In [4]:
def build_silver_nlp(spark):
    os.makedirs(NLP_SILVER, exist_ok=True)

    df = spark.read.parquet(SILVER)

    # auto select text column
    df = df.withColumn(
        "text",
        F.coalesce(
            F.col("Article"),
            F.col("Textrank_summary"),
            F.col("Lsa_summary"),
            F.col("Luhn_summary"),
            F.col("Lexrank_summary"),
            F.col("Article_title"),   # solve empty text cases
        )
    )

    # filter non-empty text
    df = df.filter(F.col("text").isNotNull() & (F.length(F.trim("text")) > 0))

    df = df.select(
        "news_id", "Date", "Article_title", "Stock_symbol",
        "Publisher", "Author", "Url", "text"
    ).repartition(200)

    df.write.mode("overwrite").parquet(NLP_SILVER)

    print("🧠 Silver_for_nlp 生成完成:", NLP_SILVER)
    return df

In [5]:
spark = create_spark()

bronze_df = build_bronze(spark)
#silver_df = build_silver(spark)
#gold_df   = build_gold(spark)
#nlp_df    = build_silver_nlp(spark)

🔥 Spark Ready: 3.3.1
📥 Loading CSV...
💾 Writing Bronze...
🥉 Bronze Ready: D:\Columbia\Fall2025\5400\project\layer\bronze
💾 Writing Silver...
🥈 Silver Ready: D:\Columbia\Fall2025\5400\project\layer\silver
💾 Writing Gold...
🥇 Gold Ready: D:\Columbia\Fall2025\5400\project\layer\gold
🧠 Silver_for_nlp 生成完成: D:\Columbia\Fall2025\5400\project\layer\silver_for_nlp


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

SILVER = r"D:\Columbia\Fall2025\5400\project\layer\silver"

df = spark.read.parquet(SILVER)

print(df.columns)

df.printSchema()

['Date', 'Article_title', 'Stock_symbol', 'Url', 'Publisher', 'Author', 'Article', 'Lsa_summary', 'Luhn_summary', 'Textrank_summary', 'Lexrank_summary', 'Publisher_norm', 'Author_norm', 'news_id']
root
 |-- Date: date (nullable = true)
 |-- Article_title: string (nullable = true)
 |-- Stock_symbol: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- Lsa_summary: string (nullable = true)
 |-- Luhn_summary: string (nullable = true)
 |-- Textrank_summary: string (nullable = true)
 |-- Lexrank_summary: string (nullable = true)
 |-- Publisher_norm: string (nullable = true)
 |-- Author_norm: string (nullable = true)
 |-- news_id: long (nullable = true)



In [7]:
import pyarrow.dataset as ds

NLP_SILVER = r"D:\Columbia\Fall2025\5400\project\layer\silver_for_nlp"
d = ds.dataset(NLP_SILVER, format="parquet")

print("NLP rows:", d.count_rows())
print("Schema:", d.schema)

NLP rows: 3242351
Schema: news_id: int64
Date: date32[day]
Article_title: string
Stock_symbol: string
Publisher: string
Author: string
Url: string
text: string
-- schema metadata --
org.apache.spark.version: '3.3.1'
org.apache.spark.sql.parquet.row.metadata: '{"type":"struct","fields":[{"' + 517


#### NLP(Silver)


In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

SILVER = r"D:\Columbia\Fall2025\5400\project\layer\silver"
BRONZE = r"D:\Columbia\Fall2025\5400\project\layer\bronze"

silver_df = spark.read.parquet(SILVER)
bronze_df = spark.read.parquet(BRONZE)

print("Bronze rows:", bronze_df.count())
print("Silver rows:", silver_df.count())
silver_df.select("Date").show(5, False)

Bronze rows: 13057514
Silver rows: 3242351
+----------+
|Date      |
+----------+
|2014-04-23|
|2012-02-15|
|2017-04-03|
|2017-02-15|
|2018-03-13|
+----------+
only showing top 5 rows



In [1]:
# distilbert
import os
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
from transformers import pipeline
import torch

BASE = r"D:\Columbia\Fall2025\5400\project\layer"
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")
SENTIMENT_SAMPLE = os.path.join(BASE, "sentiment_sample")
os.makedirs(SENTIMENT_SAMPLE, exist_ok=True)

# 1. GPU
if torch.cuda.is_available() and torch.cuda.device_count() > 0:
    device = 0
    print("✅ Using GPU device 0 for sentiment.")
else:
    device = -1
    print("⚠ 没有可用 CUDA 设备，使用 CPU。")

# 2. HF pipeline
sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=device,
)

# 3. prepare dataset
dataset = ds.dataset(NLP_SILVER, format="parquet")
total_rows = dataset.count_rows()
print("🧠 NLP total rows:", total_rows)

BATCH_ROWS = 4000        # read 4000 rows at a time from parquet
PIPELINE_BATCH = 32      # batch_size
MAX_LENGTH = 256
MAX_ROWS = 3242351
processed_rows = 0

batches = dataset.to_batches(batch_size=BATCH_ROWS)

for i, batch in enumerate(batches):
    if processed_rows >= MAX_ROWS:
        print(f"⚡ 已达到 MAX_ROWS={MAX_ROWS}，提前结束。")
        break

    table = pa.Table.from_batches([batch])
    pdf = table.to_pandas()

    # 只保留有 text 的行
    pdf = pdf[pdf["text"].notna() & (pdf["text"].astype(str).str.strip() != "")]
    if pdf.empty:
        continue

    texts = pdf["text"].astype(str).tolist()

    preds = sentiment_pipe(
        texts,
        batch_size=PIPELINE_BATCH,
        truncation=True,
        max_length=MAX_LENGTH,
    )

    pdf["sentiment_label"] = [p["label"] for p in preds]
    pdf["sentiment_score"] = [float(p["score"]) for p in preds]
    pdf["sentiment_score_signed"] = pdf["sentiment_score"].where(
        pdf["sentiment_label"] == "POSITIVE",
        -pdf["sentiment_score"],
    )

    out_pdf = pdf[["news_id", "sentiment_label",
                   "sentiment_score", "sentiment_score_signed"]]
    out_table = pa.Table.from_pandas(out_pdf, preserve_index=False)

    out_file = os.path.join(SENTIMENT_SAMPLE, f"part-{i:05d}.parquet")
    pq.write_table(out_table, out_file)

    processed_rows += len(out_pdf)
    print(f"✅ batch {i}, 当前写出 {len(out_pdf)} 行，累计 {processed_rows} 行")

print("🎉 Sentiment SAMPLE 写完:", SENTIMENT_SAMPLE)
print("实际处理行数:", processed_rows)


✅ Using GPU device 0 for sentiment.


Device set to use cuda:0


🧠 NLP total rows: 3242351
✅ batch 0, 当前写出 4000 行，累计 4000 行
✅ batch 1, 当前写出 4000 行，累计 8000 行
✅ batch 2, 当前写出 4000 行，累计 12000 行
✅ batch 3, 当前写出 4000 行，累计 16000 行
✅ batch 4, 当前写出 226 行，累计 16226 行
✅ batch 5, 当前写出 4000 行，累计 20226 行
✅ batch 6, 当前写出 4000 行，累计 24226 行
✅ batch 7, 当前写出 4000 行，累计 28226 行


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ batch 8, 当前写出 4000 行，累计 32226 行
✅ batch 9, 当前写出 224 行，累计 32450 行
✅ batch 10, 当前写出 4000 行，累计 36450 行
✅ batch 11, 当前写出 4000 行，累计 40450 行
✅ batch 12, 当前写出 4000 行，累计 44450 行
✅ batch 13, 当前写出 4000 行，累计 48450 行
✅ batch 14, 当前写出 223 行，累计 48673 行
✅ batch 15, 当前写出 4000 行，累计 52673 行
✅ batch 16, 当前写出 4000 行，累计 56673 行
✅ batch 17, 当前写出 4000 行，累计 60673 行
✅ batch 18, 当前写出 4000 行，累计 64673 行
✅ batch 19, 当前写出 222 行，累计 64895 行
✅ batch 20, 当前写出 4000 行，累计 68895 行
✅ batch 21, 当前写出 4000 行，累计 72895 行
✅ batch 22, 当前写出 4000 行，累计 76895 行
✅ batch 23, 当前写出 4000 行，累计 80895 行
✅ batch 24, 当前写出 227 行，累计 81122 行
✅ batch 25, 当前写出 4000 行，累计 85122 行
✅ batch 26, 当前写出 4000 行，累计 89122 行
✅ batch 27, 当前写出 4000 行，累计 93122 行
✅ batch 28, 当前写出 4000 行，累计 97122 行
✅ batch 29, 当前写出 222 行，累计 97344 行
✅ batch 30, 当前写出 4000 行，累计 101344 行
✅ batch 31, 当前写出 4000 行，累计 105344 行
✅ batch 32, 当前写出 4000 行，累计 109344 行
✅ batch 33, 当前写出 4000 行，累计 113344 行
✅ batch 34, 当前写出 224 行，累计 113568 行
✅ batch 35, 当前写出 4000 行，累计 117568 行
✅ batch 36, 当前写出 4000 

In [2]:
import os, pyarrow.parquet as pq

SENTIMENT_SAMPLE = r"D:\Columbia\Fall2025\5400\project\layer\sentiment_sample"
files = sorted(os.listdir(SENTIMENT_SAMPLE))
print("文件数:", len(files))
print("示例文件:", files[:5])

t = pq.read_table(os.path.join(SENTIMENT_SAMPLE, files[0]))
print("sample 一些行：")
print(t.to_pandas().head())


文件数: 1000
示例文件: ['part-00000.parquet', 'part-00001.parquet', 'part-00002.parquet', 'part-00003.parquet', 'part-00004.parquet']
sample 一些行：
         news_id sentiment_label  sentiment_score  sentiment_score_signed
0  2284922605296        POSITIVE         0.885818                0.885818
1  1967095022423        POSITIVE         0.999705                0.999705
2  2422361556929        NEGATIVE         0.984156               -0.984156
3  2181843394301        POSITIVE         0.989560                0.989560
4  1451698949482        POSITIVE         0.990559                0.990559


In [1]:
# join_sentiment_silver_sample.py
import os
from pyspark.sql import SparkSession, functions as F

def create_spark(app_name="5400-news-elt-join-sample"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark

BASE = r"D:\Columbia\Fall2025\5400\project\layer"
SILVER = os.path.join(BASE, "silver")
SENTIMENT_SAMPLE = os.path.join(BASE, "sentiment_sample")
SILVER_WITH_SENT_SAMPLE = os.path.join(BASE, "silver_with_sentiment_sample")

spark = create_spark()

silver_df = spark.read.parquet(SILVER)
sent_df = spark.read.parquet(SENTIMENT_SAMPLE)

print("银层行数：", silver_df.count())
print("情感样本行数：", sent_df.count())

# left join
silver_with_sent = silver_df.join(sent_df, on="news_id", how="left")

silver_with_sent = silver_with_sent.repartition(200, "Stock_symbol", "Date")

(
    silver_with_sent
    .write
    .mode("overwrite")
    .parquet(SILVER_WITH_SENT_SAMPLE)
)

print("✅ silver_with_sentiment_sample 写入完成：", SILVER_WITH_SENT_SAMPLE)

test_df = spark.read.parquet(SILVER_WITH_SENT_SAMPLE)
test_df.select("news_id","Article_title","sentiment_label",
               "sentiment_score_signed").show(20, False)

spark.stop()


银层行数： 3242351
情感样本行数： 3242351
✅ silver_with_sentiment_sample 写入完成： D:\Columbia\Fall2025\5400\project\layer\silver_with_sentiment_sample
+-------------+----------------------------------------------------------------------------------------------------------------------------------+---------------+----------------------+
|news_id      |Article_title                                                                                                                     |sentiment_label|sentiment_score_signed|
+-------------+----------------------------------------------------------------------------------------------------------------------------------+---------------+----------------------+
|317827582759 |Benzinga's Top Upgrades, Downgrades For July 1, 2019                                                                              |NEGATIVE       |-0.990228533744812    |
|386547061560 |Are Hedge Funds Setup For A Short Squeeze?                                                               

In [2]:
import os
from pyspark.sql import SparkSession, functions as F

def create_spark(app_name="5400-news-gold-sent"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark

BASE = r"D:\Columbia\Fall2025\5400\project\layer"
SILVER_WITH_SENT_SAMPLE = os.path.join(BASE, "silver_with_sentiment_sample")
GOLD = os.path.join(BASE, "gold")
GOLD_WITH_SENT = os.path.join(BASE, "gold_with_sentiment")

spark = create_spark()

silver_sent = spark.read.parquet(SILVER_WITH_SENT_SAMPLE)
gold_df = spark.read.parquet(GOLD)

# 1. aggregate sentiment by (Date, Stock_symbol)
sent_agg = (
    silver_sent
    .groupBy("Date", "Stock_symbol")
    .agg(
        F.avg("sentiment_score_signed").alias("avg_sentiment_score"),
        F.sum(F.when(F.col("sentiment_label") == "POSITIVE", 1).otherwise(0)).alias("pos_article_count"),
        F.sum(F.when(F.col("sentiment_label") == "NEGATIVE", 1).otherwise(0)).alias("neg_article_count"),
        # 这里 count(sentiment_label) 只统计有标签的文章（NULL 会被自动排除）
        F.count(F.col("sentiment_label")).alias("sentiment_article_count")
    )
)

# 2. join with gold
gold_with_sent = (
    gold_df.alias("g")
    .join(
        sent_agg.alias("s"),
        on=["Date", "Stock_symbol"],
        how="left"
    )
)

# 3. sentiment coverage
gold_with_sent = gold_with_sent.withColumn(
    "sentiment_coverage",
    F.when(
        F.col("article_count") > 0,
        F.col("sentiment_article_count").cast("double") / F.col("article_count")
    ).otherwise(F.lit(0.0))  # 没有文章 / 空值 时覆盖率设为 0
)

# 4. repartition
gold_with_sent = gold_with_sent.repartition(200, "Stock_symbol")

(
    gold_with_sent
    .write
    .mode("overwrite")
    .parquet(GOLD_WITH_SENT)
)

print("🏆 gold_with_sentiment 写入完成：", GOLD_WITH_SENT)

spark.stop()


🏆 gold_with_sentiment 写入完成： D:\Columbia\Fall2025\5400\project\layer\gold_with_sentiment


### Final Gold NLP


In [3]:
# gold_with_sentiment_sample.py
import os
from pyspark.sql import SparkSession, functions as F

def create_spark(app_name="5400-news-gold-sent-sample"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark

# ============================
# path
# ============================

BASE = r"D:\Columbia\Fall2025\5400\project\layer"

GOLD = os.path.join(BASE, "gold")   # 原 gold（无情感）
SILVER_WITH_SENT_SAMPLE = os.path.join(BASE, "silver_with_sentiment_sample")
GOLD_WITH_SENT_SAMPLE = os.path.join(BASE, "gold_with_sentiment_sample")

spark = create_spark()

# ============================
# read data
# ============================
gold_df = spark.read.parquet(GOLD)
silver_sent = spark.read.parquet(SILVER_WITH_SENT_SAMPLE)

print("Gold 行数（原聚合）:", gold_df.count())
print("Silver + Sentiment sample 行数:", silver_sent.count())

# ============================
# 1) aggregate sentiment by (Date, Stock_symbol)
# ============================

sent_agg = (
    silver_sent
    .groupBy("Date", "Stock_symbol")
    .agg(
        F.avg("sentiment_score_signed").alias("avg_sentiment_score"),
        F.sum(F.when(F.col("sentiment_label") == "POSITIVE", 1).otherwise(0)).alias("pos_article_count"),
        F.sum(F.when(F.col("sentiment_label") == "NEGATIVE", 1).otherwise(0)).alias("neg_article_count"),
        F.count(F.col("sentiment_label")).alias("sentiment_article_count"),
    )
)

# ratio of pos/neg
sent_agg = (
    sent_agg
    .withColumn(
        "pos_ratio",
        F.when(F.col("sentiment_article_count") > 0,
               F.col("pos_article_count") / F.col("sentiment_article_count"))
         .otherwise(F.lit(None))
    )
    .withColumn(
        "neg_ratio",
        F.when(F.col("sentiment_article_count") > 0,
               F.col("neg_article_count") / F.col("sentiment_article_count"))
         .otherwise(F.lit(None))
    )
)

print("情感聚合完成")

# ============================
# 2) join with gold
# ============================

gold_with_sent = (
    gold_df.alias("g")
    .join(sent_agg.alias("s"), on=["Date", "Stock_symbol"], how="left")
)

# 3) sentiment category
gold_with_sent = gold_with_sent.withColumn(
    "sentiment_category",
    F.when(F.col("avg_sentiment_score") >= 0.2, "POSITIVE")
     .when(F.col("avg_sentiment_score") <= -0.2, "NEGATIVE")
     .otherwise("NEUTRAL")
)

# partition
gold_with_sent = gold_with_sent.repartition(100, "Stock_symbol")

(
    gold_with_sent
    .write
    .mode("overwrite")
    .parquet(GOLD_WITH_SENT_SAMPLE)
)

print("🏆 gold_with_sentiment_sample 写入完成：", GOLD_WITH_SENT_SAMPLE)

gold_with_sent.select(
    "Date", "Stock_symbol",
    "article_count", "publisher_count", "avg_title_len",
    "avg_sentiment_score", "sentiment_article_count",
    "pos_ratio", "neg_ratio", "sentiment_category"
).show(10, False)

spark.stop()


Gold 行数（原聚合）: 1896033
Silver + Sentiment sample 行数: 3242351
情感聚合完成
🏆 gold_with_sentiment_sample 写入完成： D:\Columbia\Fall2025\5400\project\layer\gold_with_sentiment_sample
+----------+------------+-------------+---------------+-------------+-------------------+-----------------------+---------+---------+------------------+
|Date      |Stock_symbol|article_count|publisher_count|avg_title_len|avg_sentiment_score|sentiment_article_count|pos_ratio|neg_ratio|sentiment_category|
+----------+------------+-------------+---------------+-------------+-------------------+-----------------------+---------+---------+------------------+
|2010-03-03|SGOL        |1            |1              |30.0         |0.9998376369476318 |1                      |1.0      |0.0      |POSITIVE          |
|2010-04-06|LEN         |2            |2              |58.5         |-0.9546563923358917|2                      |0.0      |1.0      |NEGATIVE          |
|2010-05-13|MMSI        |2            |1              |61.0       

In [4]:
import os
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.dataset as ds

GOLD_WITH_SENT_SAMPLE = r"D:\Columbia\Fall2025\5400\project\layer\gold_with_sentiment_sample"
OUTPUT_CSV = r"D:\Columbia\Fall2025\5400\project\gold_with_sentiment_sample.csv"

dataset = ds.dataset(GOLD_WITH_SENT_SAMPLE, format="parquet")
table = dataset.to_table()
df = table.to_pandas()

df.to_csv(OUTPUT_CSV, index=False)
print("CSV 已导出：", OUTPUT_CSV)

CSV 已导出： D:\Columbia\Fall2025\5400\project\gold_with_sentiment_sample.csv


In [ ]:
# 文件名: 01_data_prep.py

import os
import shutil
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

# --- 路径定义 (需保持一致) ---
BASE = r"D:\Columbia\Fall2025\5400\project\layer"
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"
BRONZE = os.path.join(BASE, "bronze")
SILVER = os.path.join(BASE, "silver")
NLP_INPUT = os.path.join(BASE, "silver_for_nlp")

def create_spark(app_name="Step1_Prep_331"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark

def build_bronze(spark):
    # (省略实现，同原代码的 Bronze 逻辑)
    if os.path.exists(BRONZE): shutil.rmtree(BRONZE)
    df = spark.read.option("header", "true").csv(CSV_PATH)
    for c in df.columns: df = df.withColumn(c, F.trim(F.col(c)))
    df = df.withColumn("Date", F.to_date(F.col("Date"), "yyyy-mm-dd"))
    df.write.mode("overwrite").parquet(BRONZE)
    return df

def build_silver(spark):
    # (省略实现，同原代码的 Silver 逻辑)
    if os.path.exists(SILVER): shutil.rmtree(SILVER)
    bronze_df = spark.read.parquet(BRONZE)
    silver_df = bronze_df.filter(F.col("Date").isNotNull() & F.col("Stock_symbol").isNotNull()) \
                         .dropDuplicates(["Date", "Stock_symbol", "Article_title"]) \
                         .withColumn("news_id", F.md5(F.concat_ws("|", "Date", "Stock_symbol", "Article_title")))
    silver_df.write.mode("overwrite").parquet(SILVER)
    return silver_df

def export_for_nlp(spark):
    """导出 news_id 和 text，供独立 NLP 环境使用"""
    print("Step 1.3: Exporting minimal data for NLP...")
    if os.path.exists(NLP_INPUT): shutil.rmtree(NLP_INPUT)

    silver_df = spark.read.parquet(SILVER)

    nlp_df = silver_df.select(
        "news_id",
        F.concat_ws(" . ", F.col("Article_title"), F.col("Lsa_summary")).alias("text")
    ).filter("text != ''")

    # 减少分区数，方便纯 Python 环境读取（减少文件数）
    nlp_df.repartition(10).write.mode("overwrite").parquet(NLP_INPUT)
    print(f"✅ Data Prep Done! NLP input ready at: {NLP_INPUT}")

if __name__ == "__main__":
    spark = create_spark()
    build_bronze(spark)
    build_silver(spark)
    export_for_nlp(spark)
    spark.stop()

### 整理

In [9]:
import os
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

# -------------------------------------------------------------------------
# 1. 配置与初始化
# -------------------------------------------------------------------------

# CSV 路径 (这是唯一的本地文件依赖，Spark 必须从这里读取数据源)
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

# Snowflake 配置
SNOWFLAKE_CONNECTOR_PACKAGE = "net.snowflake:spark-snowflake_2.12:2.11.0-spark_3.3"
SNOWFLAKE_OPTIONS = {
    # 使用您提供的账户 ID
    "sfURL": "POBRIUX-FYC84817.snowflakecomputing.com",
    "sfUser": "AKAAAAFK",
    # ⭐⭐⭐ 认证修复: 您必须在此处提供密码或私钥文件路径 ⭐⭐⭐
    "sfPassword": "<请在此处替换为您的 Snowflake 密码>",
    # 或者如果使用 Key-Pair 认证: "pem_private_key": "/path/to/your/rsa_key.p8"

    # 使用您提供的数据库、Schema、仓库和角色
    "sfDatabase": "SNOWFLAKE_LEARNING_DB",
    "sfSchema": "PUBLIC",
    "sfWarehouse": "COMPUTE_WH",
    "sfRole": "ACCOUNTADMIN" # 增加了 Role 参数
}

# Snowflake Tables/Stage Names
BRONZE_TABLE = "BRONZE_NEWS"
SILVER_TABLE = "SILVER_NEWS"
NLP_INPUT_STAGE = "@NEWS_NLP_INPUT_STAGE" # Snowflake 内部 Stage，用于 NLP 输入


def create_spark(app_name="5400-news-elt-snowflake"):
    """初始化 Spark Session 并加载 Snowflake Connector JAR"""
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.jars.packages", SNOWFLAKE_CONNECTOR_PACKAGE)
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark


def save_to_snowflake(df, table_name, mode="overwrite", is_stage=False):
    """写入 Snowflake Table 或 Stage"""
    print(f"❄️ 正在尝试将 DataFrame 写入 Snowflake {'Stage' if is_stage else 'Table'}: {table_name}...")
    try:
        writer = df.write.format("net.snowflake.spark.snowflake").options(**SNOWFLAKE_OPTIONS)
        if is_stage:
            # 写入 Stage (Spark Connector 自动使用 COPY INTO 流程)
            writer.option("dbtable", table_name).option("autopurge", "true").save()
        else:
            # 写入 Table
            writer.option("dbtable", table_name).mode(mode).save()

        print(f"✅ 数据已成功写入 Snowflake {'Stage' if is_stage else 'Table'}: {table_name}")
    except Exception as e:
        print(f"❌ Snowflake 写入失败。错误: {e}")

In [10]:
def build_bronze(spark, csv_path):
    """读取 CSV，直接写入 BRONZE_NEWS 表"""
    print("🚀 1. Building Bronze Layer...")
    # 保持原代码的 CSV 读取逻辑
    df = spark.read.format("csv").options(header="true", inferSchema="true", multiLine="true", escape="\"", quote="\"", mode="PERMISSIVE").load(csv_path)

    # 基础清洗
    for c, t in df.dtypes:
        if t == "string":
            df = df.withColumn(c, F.trim(F.col(c)))

    if "Date" in df.columns:
        df = df.withColumn("Date", F.to_date("Date", "yyyy-MM-dd"))

    df = df.repartition(400)
    save_to_snowflake(df, BRONZE_TABLE, mode="overwrite")
    print("🥉 Bronze Ready.")
    return df


def build_silver(spark):
    """读取 BRONZE_NEWS，清洗后写入 SILVER_NEWS 表"""
    print("🚀 2. Building Silver Layer...")
    # 从 Snowflake 读取 Bronze
    df = spark.read.format("net.snowflake.spark.snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable", BRONZE_TABLE).load()

    # 清洗和去重
    if "Date" in df.columns: df = df.filter(F.col("Date").isNotNull())
    if "Stock_symbol" in df.columns: df = df.filter(F.col("Stock_symbol").isNotNull())
    df = df.dropDuplicates()
    if "Publisher" in df.columns: df = df.withColumn("Publisher_norm", F.upper("Publisher"))
    if "Author" in df.columns: df = df.withColumn("Author_norm", F.upper("Author"))
    df = df.withColumn("news_id", F.monotonically_increasing_id())
    df = df.repartition(400, "Stock_symbol", "Date")

    save_to_snowflake(df, SILVER_TABLE, mode="overwrite")
    print("🥈 Silver Ready.")
    return df


def build_silver_nlp(spark):
    """准备 NLP 输入：合并文本源，筛选非空文本，写入 Snowflake Stage"""
    print("🚀 3. Preparing Data for NLP (Writing to Snowflake Stage)...")
    # 从 Snowflake 读取 Silver
    df = spark.read.format("net.snowflake.spark.snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable", SILVER_TABLE).load()

    # 文本合并逻辑
    df = df.withColumn(
        "text",
        F.coalesce(F.col("Article"), F.col("Textrank_summary"), F.col("Lsa_summary"), F.col("Luhn_summary"), F.col("Lexrank_summary"), F.col("Article_title"))
    )

    df = df.filter(F.col("text").isNotNull() & (F.length(F.trim("text")) > 0))

    df = df.select("news_id", "text").repartition(200)

    # ⭐ 写入 Snowflake Stage
    save_to_snowflake(df, NLP_INPUT_STAGE, is_stage=True)

    print("🧠 Silver_for_nlp Stage 文件已生成:", NLP_INPUT_STAGE)
    return df

In [ ]:
# -------------------------------------------------------------------------
# 3. 主执行块
# -------------------------------------------------------------------------

if __name__ == "__main__":
    spark = create_spark()

    # 按照顺序执行所有 ETL 步骤，Silver 和 Gold 会自动写入 Snowflake
    bronze_df = build_bronze(spark, CSV_PATH)
    # silver_df = build_silver(spark)
    # gold_df   = build_gold(spark)
    # nlp_df    = build_silver_nlp(spark)

    spark.stop()
    print("✅ ETL 流程完成，数据已同步至 Snowflake，NLP 输入已准备就绪。")

🚀 1. Building Bronze Layer...
❄️ 正在尝试将 DataFrame 写入 Snowflake Table: BRONZE_NEWS...


In [ ]:
# 文件名: etl_layers.py (已修复连接参数和认证缺失问题)

import os
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

# -------------------------------------------------------------------------
# 1. 配置与初始化
# -------------------------------------------------------------------------

# CSV 路径 (Spark 必须读取本地文件)
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

# Snowflake 配置 (使用您提供的参数)
SNOWFLAKE_CONNECTOR_PACKAGE = "net.snowflake:spark-snowflake_2.12:2.11.0-spark_3.3"
SNOWFLAKE_OPTIONS = {
    # 使用您提供的账户 ID
    "sfURL": "POBRIUX-FYC84817.snowflakecomputing.com",
    "sfUser": "AKAAAAFK",
    # ⭐⭐⭐ 认证修复: 您必须在此处提供密码或私钥文件路径 ⭐⭐⭐
    "sfPassword": "<请在此处替换为您的 Snowflake 密码>",
    # 或者如果使用 Key-Pair 认证: "pem_private_key": "/path/to/your/rsa_key.p8"

    # 使用您提供的数据库、Schema、仓库和角色
    "sfDatabase": "SNOWFLAKE_LEARNING_DB",
    "sfSchema": "PUBLIC",
    "sfWarehouse": "COMPUTE_WH",
    "sfRole": "ACCOUNTADMIN" # 增加了 Role 参数
}

# Snowflake Tables/Stage Names
BRONZE_TABLE = "BRONZE_NEWS"
SILVER_TABLE = "SILVER_NEWS"
NLP_INPUT_STAGE = "@NEWS_NLP_INPUT_STAGE"


def create_spark(app_name="5400-news-elt-snowflake"):
    """初始化 Spark Session 并加载 Snowflake Connector JAR"""
    # ... (函数体不变) ...
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.jars.packages", SNOWFLAKE_CONNECTOR_PACKAGE)
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark


def save_to_snowflake(df, table_name, mode="overwrite", is_stage=False):
    """写入 Snowflake Table 或 Stage"""
    print(f"❄️ 正在尝试将 DataFrame 写入 Snowflake {'Stage' if is_stage else 'Table'}: {table_name}...")
    try:
        writer = df.write.format("net.snowflake.spark.snowflake").options(**SNOWFLAKE_OPTIONS)
        if is_stage:
            writer.option("dbtable", table_name).option("autopurge", "true").save()
        else:
            writer.option("dbtable", table_name).mode(mode).save()

        print(f"✅ 数据已成功写入 Snowflake {'Stage' if is_stage else 'Table'}: {table_name}")
    except Exception as e:
        print(f"❌ Snowflake 写入失败。错误: {e}")
        raise # 重新抛出错误，以便您能看到详细的连接失败信息


def build_bronze(spark, csv_path):
    """读取 CSV，直接写入 BRONZE_NEWS 表"""
    print("🚀 1. Building Bronze Layer...")
    # ... (CSV 读取和基础清洗逻辑不变) ...
    df = spark.read.format("csv").options(header="true", inferSchema="true", multiLine="true", escape="\"", quote="\"", mode="PERMISSIVE").load(csv_path)

    for c, t in df.dtypes:
        if t == "string":
            df = df.withColumn(c, F.trim(F.col(c)))

    if "Date" in df.columns:
        df = df.withColumn("Date", F.to_date("Date", "yyyy-MM-dd"))

    df = df.repartition(400)
    save_to_snowflake(df, BRONZE_TABLE, mode="overwrite")
    print("🥉 Bronze Ready.")
    return df


def build_silver(spark):
    """读取 BRONZE_NEWS，清洗后写入 SILVER_NEWS 表"""
    # ... (读取和清洗逻辑不变) ...
    print("🚀 2. Building Silver Layer...")
    df = spark.read.format("net.snowflake.spark.snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable", BRONZE_TABLE).load()

    if "Date" in df.columns: df = df.filter(F.col("Date").isNotNull())
    if "Stock_symbol" in df.columns: df = df.filter(F.col("Stock_symbol").isNotNull())
    df = df.dropDuplicates()
    if "Publisher" in df.columns: df = df.withColumn("Publisher_norm", F.upper("Publisher"))
    if "Author" in df.columns: df = df.withColumn("Author_norm", F.upper("Author"))
    df = df.withColumn("news_id", F.monotonically_increasing_id())
    df = df.repartition(400, "Stock_symbol", "Date")

    save_to_snowflake(df, SILVER_TABLE, mode="overwrite")
    print("🥈 Silver Ready.")
    return df


def build_silver_nlp(spark):
    """准备 NLP 输入：写入 Snowflake Stage"""
    # ... (逻辑不变) ...
    print("🚀 3. Preparing Data for NLP (Writing to Snowflake Stage)...")
    df = spark.read.format("net.snowflake.spark.snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable", SILVER_TABLE).load()

    df = df.withColumn(
        "text",
        F.coalesce(F.col("Article"), F.col("Textrank_summary"), F.col("Lsa_summary"), F.col("Luhn_summary"), F.col("Lexrank_summary"), F.col("Article_title"))
    )

    df = df.filter(F.col("text").isNotNull() & (F.length(F.trim("text")) > 0))

    df = df.select("news_id", "text").repartition(200)

    save_to_snowflake(df, NLP_INPUT_STAGE, is_stage=True)

    print("🧠 Silver_for_nlp Stage 文件已生成:", NLP_INPUT_STAGE)
    return df


# -------------------------------------------------------------------------
# 3. 主执行块
# -------------------------------------------------------------------------

if __name__ == "__main__":
    spark = create_spark()

    # 按照顺序执行 ETL 步骤
    try:
        build_bronze(spark, CSV_PATH)
        # build_silver(spark)
       #  build_silver_nlp(spark)
    except Exception as e:
        print(f"\n致命错误：ETL 流程中断。请检查您的 Snowflake 凭证。")

    spark.stop()
    print("✅ ETL 流程执行完毕。")

In [ ]:
# 文件名: nlp_inference_batch.py (独立 NLP 推理)
# 运行环境要求: Python, pandas, pyarrow, torch, transformers (无需 PySpark)

import os
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
from transformers import pipeline
import torch
import shutil

# --- 路径定义 (需保持一致) ---
BASE = r"D:\Columbia\Fall2025\5400\project\layer"
NLP_SILVER = os.path.join(BASE, "silver_for_nlp") # ETL 脚本的输出
SENTIMENT_SAMPLE = os.path.join(BASE, "sentiment_sample") # NLP 脚本的输出

def run_nlp_inference():
    """执行独立、高性能的 NLP 情感分析批处理"""
    print("🚀 Step 2: Starting NLP Inference (Pure Python/AI Environment)...")

    # 1. 路径准备和 GPU 检测
    os.makedirs(SENTIMENT_SAMPLE, exist_ok=True)
    if os.path.exists(SENTIMENT_SAMPLE): shutil.rmtree(SENTIMENT_SAMPLE)

    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        device = 0
        print("✅ Using GPU device 0 for sentiment.")
    else:
        device = -1
        print("⚠ 没有可用 CUDA 设备，使用 CPU。")

    # 2. Hugging Face pipeline 初始化
    try:
        sentiment_pipe = pipeline(
            "sentiment-analysis",
            model="distilbert-base-uncased-finetuned-sst-2-english",
            device=device,
        )
    except Exception as e:
        print(f"❌ 模型加载失败。请检查环境和网络。错误: {e}")
        return

    # 3. 准备数据集和处理参数
    try:
        dataset = ds.dataset(NLP_SILVER, format="parquet")
        total_rows = dataset.count_rows()
    except Exception as e:
        print(f"❌ 无法读取 NLP 输入文件。请确保 etl_layers.py 已运行。错误: {e}")
        return

    print("🧠 NLP total rows:", total_rows)

    BATCH_ROWS = 4000        # 每次从 Parquet 读取的行数
    PIPELINE_BATCH = 32      # 模型推理的内部 batch_size
    MAX_LENGTH = 256
    MAX_ROWS = total_rows    # 您的原代码设置为全量
    processed_rows = 0

    batches = dataset.to_batches(batch_size=BATCH_ROWS)

    # 4. 批处理循环和推理
    for i, batch in enumerate(batches):
        if processed_rows >= MAX_ROWS:
            print(f"⚡ 已达到 MAX_ROWS={MAX_ROWS}，提前结束。")
            break

        table = pa.Table.from_batches([batch])
        pdf = table.to_pandas()

        # 只保留有 text 的行
        pdf = pdf[pdf["text"].notna() & (pdf["text"].astype(str).str.strip() != "")]
        if pdf.empty:
            continue

        texts = pdf["text"].astype(str).tolist()

        preds = sentiment_pipe(
            texts,
            batch_size=PIPELINE_BATCH,
            truncation=True,
            max_length=MAX_LENGTH,
        )

        # 整理和计算分数
        pdf["sentiment_label"] = [p["label"] for p in preds]
        pdf["sentiment_score"] = [float(p["score"]) for p in preds]
        pdf["sentiment_score_signed"] = pdf["sentiment_score"].where(
            pdf["sentiment_label"] == "POSITIVE",
            -pdf["sentiment_score"],
        )

        # 写入结果 (仅保留 news_id 和分数)
        out_pdf = pdf[["news_id", "sentiment_label",
                       "sentiment_score", "sentiment_score_signed"]]
        out_table = pa.Table.from_pandas(out_pdf, preserve_index=False)

        out_file = os.path.join(SENTIMENT_SAMPLE, f"part-{i:05d}.parquet")
        pq.write_table(out_table, out_file)

        processed_rows += len(out_pdf)
        print(f"✅ batch {i}, 当前写出 {len(out_pdf)} 行，累计 {processed_rows} 行")

    print("🎉 Sentiment SAMPLE 写完:", SENTIMENT_SAMPLE)
    print("实际处理行数:", processed_rows)

if __name__ == "__main__":
    run_nlp_inference()